Yesterday, I tried to combine Python and R into the same notebook, with mixed results (leaning towards "not good enough").  However, today I realized I could use IPython magic to run R (magic shown in comment since Quarto auto-removes it for final display^[Which I like, but is annoying for this specific task.  There's probably a Quarto setting to turn it off, if I look hard enough for it.]):

In [33]:
# Proof that I'm in a python environment
import sys
sys.__version__

sys.version_info(major=3, minor=9, micro=13, releaselevel='final', serial=0)

In [26]:
%%script R --no-save
#%%script R --no-save
print(c("Hello", "World"))


R version 4.2.1 (2022-06-23) -- "Funny-Looking Kid"
Copyright (C) 2022 The R Foundation for Statistical Computing
Platform: x86_64-apple-darwin13.4.0 (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> #%%script R --no-save
> print(c("Hello", "World"))
[1] "Hello" "World"
> 


Unfortunately this comes with a lot of fluff, so I'll need to figure out how to get rid of it!  Also, it syntax highlights according to Python, although that isn't too big a deal.

I found that, with `rpy2` installed, we can get a simple `%%R` cell magic; however I have trouble installing `rpy2` because I'm on an M1 Mac so things are weird (for example, note the platform output in:
```
R version 4.2.1 (2022-06-23) -- "Funny-Looking Kid"
Copyright (C) 2022 The R Foundation for Statistical Computing
Platform: x86_64-apple-darwin13.4.0 (64-bit)
```

By specifying `--no-echo`, however, we get rid of most of the extra fluff!

In [27]:
%%script R --no-save --no-echo
#%%script R --no-save --no-echo
print("Hello")
print("World")
print(c("Nice", "To", "Meet", "You"))

[1] "Hello"
[1] "World"
[1] "Nice" "To"   "Meet" "You" 


However, with this setup variables are not saved:

In [28]:
%%script R --no-save --no-echo
#%%script R --no-save --no-echo
x <- 5
print(x)

[1] 5


In [29]:
%%script R --no-save --no-echo
#%%script R --no-save --no-echo
tryCatch(x, error=function(cond){"X not found!"})

[1] "X not found!"


This is not a problem if everything can be done in one cell before continuing with python.  I should note also that in an R notebook, we could still run Python:

In [30]:
%%python3
#%%python3
print("Hello")
print("World")

Hello
World


There are 3 flaws with this approach:
* Syntax highlighting not correct
  - No big deal, highlighting R as if it were Pyhon should be mostly fine
* No persistent variables
  - This could be pretty annoying for some tasks
* Need both R and Python in the same conda environment
  - This is a big deal as they have mutual dependencies that they fight over, preventing `conda` installation of Bioconductor packages (as far as I can tell; my diagnosis might not be perfect)

To fix the last flaw, I could potentially write a bash script that handles environment activation and deactivation, and to fix the middle flaw I could potentially save / load `.Rdata` files in such a script as well.  However, I'd rather not have to write my own bash script - they scare me 😅.